In [2]:
!pip install plotly

     |████████████████████████████████| 7.1 MB 10.2 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=2672bf5a1fd494e7418fae5829cc18b36085bb57bd6aa8b9bbfc659022fcc1d9
  Stored in directory: /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [1]:
import pandas as pd
import glob
path=r'/home/jovyan/work/rlAgentTest/results'
all_files = glob.glob(path + "/*.csv")
filenames=list()
disKDA_df_dict=dict();uniKDA_df_dict=dict();weightAVG_df_dict=dict()
for filename in all_files:
    df = pd.read_csv(filename, index_col='Unnamed: 0', header=0)
    df= df.rename(columns=lambda x: x.lower().replace('-',''))
    filename=filename.split('/')[-1].split('.')[0]
    filenames.append(filename)
    if('disKDA' in filename):disKDA_df_dict.update({filename: df})
    if('uniKDA' in filename):uniKDA_df_dict.update({filename: df})
    if('weightedAVG' in filename):weightAVG_df_dict.update({filename: df})

In [4]:
list(uniKDA_df_dict.keys())

['cham3_wins_LnMlp_uniKDA',
 'cham4pv_rewards_LnMlp_uniKDA',
 'cham1pv_wins_Mlp_uniKDA',
 'cham3pv_rewards_LnMlp_uniKDA',
 'cham4pv_rewards_Mlp_uniKDA',
 'cham4_rewards_LnMlp_uniKDA',
 'cham3pv_wins_Mlp_uniKDA',
 'cham1_wins_LnMlp_uniKDA',
 'cham2pv_wins_LnMlp_uniKDA',
 'cham2pv_rewards_LnMlp_uniKDA',
 'cham1pv_wins_LnMlp_uniKDA',
 'cham5pv_wins_Mlp_uniKDA',
 'cham1_rewards_LnMlp_uniKDA',
 'random_wins_uniKDA',
 'cham1pv_rewards_Mlp_uniKDA',
 'cham5_rewards_LnMlp_uniKDA',
 'cham5pv_rewards_Mlp_uniKDA',
 'cham5pv_wins_LnMlp_uniKDA',
 'cham5_wins_LnMlp_uniKDA',
 'random_rewards_uniKDA',
 'cham5pv_rewards_LnMlp_uniKDA',
 'cham2_wins_LnMlp_uniKDA',
 'cham3pv_wins_LnMlp_uniKDA',
 'cham3pv_rewards_Mlp_uniKDA',
 'cham3_rewards_LnMlp_uniKDA',
 'cham1_rewards_Mlp_uniKDA',
 'cham2pv_wins_Mlp_uniKDA',
 'cham2_rewards_Mlp_uniKDA',
 'cham5_rewards_Mlp_uniKDA',
 'cham1_wins_Mlp_uniKDA',
 'cham4_rewards_Mlp_uniKDA',
 'cham2_wins_Mlp_uniKDA',
 'cham4pv_wins_Mlp_uniKDA',
 'cham3_wins_Mlp_uniKDA',
 'cha

In [3]:
reorder_cols=['cham1_LnMlp', 'cham1_Mlp', 'cham1_random',
           'cham2_LnMlp', 'cham2_Mlp', 'cham2_random',
           'cham3_LnMlp', 'cham3_Mlp', 'cham3_random', 
           'cham4_LnMlp', 'cham4_Mlp', 'cham4_random',
           'cham5_LnMlp', 'cham5_Mlp', 'cham5_random', 
           'cham1pv_LnMlp', 'cham1pv_Mlp', 'cham1pv_random', 
           'cham2pv_LnMlp', 'cham2pv_Mlp', 'cham2pv_random',
           'cham3pv_LnMlp','cham3pv_Mlp', 'cham3pv_random', 
           'cham4pv_LnMlp', 'cham4pv_Mlp', 'cham4pv_random',
           'cham5pv_LnMlp','cham5pv_Mlp', 'cham5pv_random']

In [64]:
def getMeans(df_dict):
    random_dict = { your_key: df_dict[your_key] for your_key in df_dict.keys() if 'random' in your_key}
    agent_dict = { your_key: df_dict[your_key] for your_key in df_dict.keys() if 'random' not in your_key}
    for name,df in random_dict.items():
        random_type=name.split('_')[0]
        df=df.add_suffix('_'+random_type)
        if('wins' in name):
            wins_df=df
        if('rewards' in name):
            rewards_df=df

    for name,df in agent_dict.items():
        agent_name=name.split('_')[0]
        model_name=name.split('_')[2]
        df=df.add_suffix('_'+model_name)
        if('wins' in name):
            col = df[agent_name+'_'+model_name]
            wins_df=wins_df.join(col)
#             print(col)
        if('rewards' in name):
            col = df[agent_name+'_'+model_name]
            rewards_df=rewards_df.join(col)
    rewards_summarize = rewards_df[reorder_cols].describe()
    wins_summarize = wins_df[reorder_cols].describe()
    lnMLP_rewards_means=rewards_summarize.filter(regex='_LnMlp').loc[['mean'],:].values[0]
    MLP_rewards_means=rewards_summarize.filter(regex='_Mlp').loc[['mean'],:].values[0]
    random_rewards_means=rewards_summarize.filter(regex='_random').loc[['mean'],:].values[0]
    lnMLP_wins_means=wins_summarize.filter(regex='_LnMlp').loc[['mean'],:].values[0]
    MLP_wins_means=wins_summarize.filter(regex='_Mlp').loc[['mean'],:].values[0]
    
    random_wins_means=wins_summarize.filter(regex='_random').loc[['mean'],:].values[0]
    return {'rewards':[lnMLP_rewards_means, MLP_rewards_means, random_rewards_means],
            'wins':[lnMLP_wins_means,MLP_wins_means,random_wins_means]}
    

In [69]:
results=getMeans(uniKDA_df_dict)
lnMLP_rewards_means, MLP_rewards_means, random_rewards_means=results['rewards']
lnMLP_wins_means,MLP_wins_means,random_wins_means=results['wins']

In [70]:
import plotly.graph_objects as go

player=['Building 1','Building 2','Building 3','Building 4','Building 5','Building 1 PV','Building 2 PV','Building 3 PV','Building 4 PV','Building 5 PV']

fig = go.Figure()
fig.add_trace(go.Bar(x=player,
                y=MLP_rewards_means,
                text=MLP_rewards_means,
                name='MLP',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=player,
                y=lnMLP_rewards_means,
                text=lnMLP_rewards_means,
                name='LnMLP',
                marker_color='rgb(26, 118, 255)'
                ))

fig.add_trace(go.Bar(x=player,
                y=random_rewards_means,
                text=random_rewards_means,
                name='random',
                marker_color='rgb(8,48,107)'
                ))


fig.update_layout(
    height=500,
    title='RL agent average rewards vs random agent comparison in DisKDA',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Reward(THB)',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

fig.show()

In [71]:

fig = go.Figure()
fig.add_trace(go.Bar(x=player,
                y=[x*100 for x in MLP_wins_means],
                text=[x*100 for x in MLP_wins_means],
                name='MLP',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=player,
                y=[x*100 for x in lnMLP_wins_means],
                text=[x*100 for x in lnMLP_wins_means],
                name='LnMLP',
                marker_color='rgb(26, 118, 255)'
                ))

fig.add_trace(go.Bar(x=player,
                y=[x*100 for x in random_wins_means],
                text=[x*100 for x in random_wins_means],
                name='random',
                marker_color='rgb(8,48,107)'
                ))

fig.update_layout(
    height=500,
    title='%Winning RL agent vs random agent comparison in DisKDA',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Percentage Win',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.update_traces(texttemplate='%{text:.2s}%', textposition='inside')
fig.show()